In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml

from aif360.datasets import StandardDataset

data = fetch_openml(data_id=1590, as_frame=True)

X_raw = data.data
y_raw = data.target

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

In [2]:
X_unstandardised =  pd.get_dummies(X_raw)
y = (y_raw == ">50K") * 1

In [3]:
# preprocess_data
SCALER = MinMaxScaler()
X = SCALER.fit_transform(X_unstandardised)
X = pd.DataFrame(X, columns=X_unstandardised.columns)

In [4]:
protected_attribute_name = f"sex_Male"

privileged_groups = [{protected_attribute_name: 1}]
unprivileged_groups = [{protected_attribute_name: 0}]

def create_dataset(
    X: pd.DataFrame, y, protected_attribute_name: str
) -> StandardDataset:
    if isinstance(y, np.ndarray):
        y = pd.Series(y.flatten(), index=X.index, name='class')
    return StandardDataset(
        df=pd.concat([X, y], axis=1),
        label_name="class",
        favorable_classes=[1],
        protected_attribute_names=[protected_attribute_name],
        privileged_classes=[[1]],
    )

# Step 1: Create the StandardDataset object
dataset = create_dataset(X, y, protected_attribute_name)

# Step 2: train-test-val :: 15:3:3
dataset_train, dataset_test = dataset.split([0.7], shuffle=True, seed=42)

In [50]:
from aif360.algorithms.preprocessing import DisparateImpactRemover

DI = DisparateImpactRemover(repair_level=1)
dataset_train_transf = DI.fit_transform(dataset_train)
dataset_test_transf = DI.fit_transform(dataset_test)

In [41]:
LR = LogisticRegression(solver="liblinear", random_state=42)
LR.fit(dataset_train.features, dataset_train.labels)
y_test_pred = LR.predict(dataset_test.features)



63

In [37]:
train_repd = DI.fit_transform(dataset_train)
test_repd = DI.fit_transform(dataset_test)

LogisticRegression()

In [43]:
import numpy as np
from aif360.metrics import BinaryLabelDatasetMetric


index = dataset_train.feature_names.index('sex_Male')

di = DisparateImpactRemover(repair_level=1)
train_repd = di.fit_transform(dataset_train)
test_repd = di.fit_transform(dataset_test)

X_tr = np.delete(train_repd.features, index, axis=1)
X_te = np.delete(test_repd.features, index, axis=1)
y_tr = train_repd.labels.ravel()

lmod = LogisticRegression(class_weight='balanced', solver='liblinear')
lmod.fit(X_tr, y_tr)

test_repd_pred = test_repd.copy()
test_repd_pred.labels = lmod.predict(X_te)

p = [{'sex_Male': 1}]
u = [{'sex_Male': 0}]

cm = BinaryLabelDatasetMetric(test_repd_pred, privileged_groups=p, unprivileged_groups=u)
DIs = (cm.disparate_impact())

In [44]:
np.delete(train_repd.features, index, axis=1).shape

(23932, 104)

In [45]:
train_repd.features.shape

(23932, 105)

In [40]:
DI.fit_predi(dataset_test)

NotImplementedError: 'predict' is not supported for this class. Perhaps you meant 'transform' or 'fit_predict' instead?

In [25]:
train_repd.convert_to_dataframe()[0]

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,class
12623,0.356164,0.102108,0.800000,0.050601,0.0,0.336735,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
31614,0.027397,0.144109,0.533333,0.000000,0.0,0.397959,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
26030,0.465753,0.125381,0.933333,0.033250,0.0,0.336735,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
17448,0.068493,0.080703,0.600000,0.000000,0.0,0.071429,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
36093,0.438356,0.250292,0.533333,0.000000,0.0,0.602041,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14545,0.219178,0.095792,0.533333,0.000000,0.0,0.336735,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
39936,0.136986,0.151671,0.800000,0.000000,0.0,0.397959,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
36262,0.369863,0.135259,0.800000,0.000000,0.0,0.295918,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9330,0.575342,0.222001,0.600000,0.000000,0.0,0.469388,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [26]:
dataset_train.convert_to_dataframe()[0]

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,class
12623,0.356164,0.107435,0.800000,0.051781,0.0,0.397959,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
31614,0.027397,0.144109,0.533333,0.000000,0.0,0.397959,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
26030,0.465753,0.133302,0.933333,0.033250,0.0,0.397959,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
17448,0.068493,0.080703,0.600000,0.000000,0.0,0.071429,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
36093,0.438356,0.267374,0.533333,0.000000,0.0,0.704082,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14545,0.219178,0.100912,0.533333,0.000000,0.0,0.397959,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
39936,0.136986,0.151671,0.800000,0.000000,0.0,0.397959,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
36262,0.369863,0.147693,0.800000,0.000000,0.0,0.346939,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9330,0.575342,0.234394,0.600000,0.000000,0.0,0.551020,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [27]:
dataset_train.convert_to_dataframe()[0] == train_repd.convert_to_dataframe()[0]

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,class
12623,True,False,True,False,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
31614,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
26030,True,False,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
17448,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
36093,True,False,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14545,True,False,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
39936,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
36262,True,False,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9330,True,False,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [35]:
print(train_repd.labels.shape[0])
print(train_repd.convert_to_dataframe()[0].equals(dataset_train.convert_to_dataframe()[0]))
print((train_repd.labels == dataset_train.labels).sum() == train_repd.labels.shape[0])

23932
False
True
